In [ ]:
# Change these

locality = "nc-guilford"
verbose = True
clear_checkpoints = False
clear_model_results = False

# 1. Basic setup

In [ ]:
from init_notebooks import setup_environment
setup_environment()

In [ ]:
# import a bunch of stuff

import os
import pandas as pd
from openavmkit.pipeline import (
    NotebookState, 
    set_locality,
    examine_df
)
from openavmkit.utilities.settings import (
    load_settings
)
from openavmkit.checkpoint import (
    from_checkpoint,
    read_checkpoint,
    write_checkpoint,
    delete_checkpoints
)
from openavmkit.benchmark import (
    run_models, 
    MultiModelResults
)

In [ ]:
if 'inited' not in globals():
    nbs: NotebookState = None
    inited = True
nbs = set_locality(nbs, locality)
settings = load_settings()

In [ ]:
if clear_checkpoints:
    delete_checkpoints("1-predict")

In [ ]:
# load the data
df = read_checkpoint("1-predict-03-out")

In [ ]:
results : MultiModelResults = run_models(
    df,
    settings,
    use_saved_results=(not clear_model_results),
    verbose=verbose
)